In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [3]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [5]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-data_test.pkl',
 'molecule-with-openbabel_test.pkl',
 'eachtype_te

In [6]:
mytrial = []

In [ ]:
# 'scalar_coupling_constant', 'fc', 'sd','pso','dso'
for y in ['pso']:
    for t in np.arange(0,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=pso, type=0


In [11]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['pso']

In [12]:
columns_list = []

In [13]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [18]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-16 11:18:23.406847,modeling for 0,78,0.820160,0.933006,0.112845,modeling for 0
1,2019-07-16 11:18:40.952761,modeling for 3,59,0.161547,0.200190,0.038643,modeling for 3
2,2019-07-16 11:18:43.612953,modeling for 1,58,0.286269,0.433112,0.146842,modeling for 1
3,2019-07-16 11:18:50.776218,modeling for 4,59,0.154898,0.237684,0.082786,modeling for 4
4,2019-07-16 11:19:50.783660,modeling for 2,68,0.360945,0.391322,0.030378,modeling for 2
5,2019-07-16 11:20:19.215675,modeling for 6,59,0.196629,0.225852,0.029223,modeling for 6
6,2019-07-16 11:21:38.304023,modeling for 5,59,0.427043,0.453386,0.026343,modeling for 5
7,2019-07-16 11:21:46.386701,modeling for 7,49,0.102380,0.141882,0.039502,modeling for 7


In [16]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.44499493261917566
0 0.9330056960257216
3 0.2001904951609826
1 0.4331118692460489
4 0.23768410461908276
2 0.39132241115217037
6 0.22585247088385893
5 0.4533857586621615
7 0.14188187674382496


In [22]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.210720,0.871862,0.746697,1.206203,1.007102
2505538,7163685,3.809454,3.867529,2.588954,2.918856,3.701579
2505539,7163686,3.658385,3.428395,3.517420,1.978553,3.407225
2505540,7163687,4.435589,4.727233,5.057265,4.668085,4.264797
2505541,7163688,118.513458,118.589493,116.857854,117.165564,112.934275


In [17]:
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['sd'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_pickle('../../data/oof/lgbm_pso_test.pkl')

In [18]:
df_submit = df_valid_pred.rename(columns={'predict':'sd', 'index':'id'})
df_submit.to_pickle('../../data/oof/lgbm_pso_train.pkl')